In [1]:
import sys
import cv2
sys.path.append('../')
from utils.utils import read_from_env_file
from utils.dataset_creator_utils import create_ds_from_video_feed
env_vars = read_from_env_file()

In [2]:
create_ds_from_video_feed(feed=env_vars["URL_CAMERA_TIENDA"],num_images=100)

100%|██████████| 100/100 [2:51:54<00:00, 103.14s/it] 
